<a href="https://colab.research.google.com/github/santiagoprado12/Tesis/blob/main/lstm_5_dias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
from math import sqrt
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import plotly.offline as py
import plotly.graph_objects as go
import statistics as stats

In [ ]:
def split_data(test_data=1, step=1, datos=None, train_min=1):
  n=train_min
  i=1
  train_index=[]
  test_index=[]
  while (test_data+n)<(len(datos)):
    train_index.append(list(range(0,n)))
    test_index.append(list(range(n,n+test_data)))
    n+=step
  train_index.append(list(range(0,len(datos)-test_data)))
  test_index.append(list(range(len(datos)-test_data,len(datos))))
  return train_index, test_index

In [ ]:
# leer dataset
dataset = pd.read_csv(r'Casos diarios y Beta.csv', index_col= 'fechas', usecols=['activos por dia', 'fechas']).values
# escalar dataset entre (-1,1)
scaler = MinMaxScaler(feature_range = (-1, 1))
data_scaled = scaler.fit_transform(dataset)

In [ ]:
train_index, test_index = split_data(test_data=5, step=7, datos=data_scaled, train_min=30)

In [ ]:
np.random.seed(0)

In [ ]:
medias = []
des_std = []

rmses = []

neuronas = [15, 20]
epocas = [100, 120, 140, 160, 180]

for n in neuronas:
  for e in epocas:
    print("###############################################################"  )
    print("###############################################################"  )
    print("INICIANDO PRUEBAS CON", n, "neuronas y", e )
    print("###############################################################"  )
    print("###############################################################"  )
    for i,test in enumerate(test_index):
      data_train_scaled, data_test_scaled = data_scaled[train_index[i]], data_scaled[test_index[i]]

      x_train = []
      y_train = []
      long_seq = 5

      for i in range(0,len(data_train_scaled)-long_seq): 
        x_train.append(data_train_scaled[i:(i+long_seq),0])
        y_train.append(data_train_scaled[i+long_seq,0])

      x_train = np.array(x_train)
      y_train = np.array(y_train)

      x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

      modelo = Sequential()
      modelo.add(LSTM(units=n, input_shape=(x_train.shape[1],1), activation='tanh'))
      modelo.add(Dense(units=1))
      modelo.compile(optimizer='adam', loss='mse')
      modelo.fit(x_train,y_train,epochs=e,batch_size=5)

      inputs = data_scaled[len(data_train_scaled)-long_seq:(len(data_train_scaled)+len(data_test_scaled))]
      x_test = []
      for i in range(0,len(inputs)-long_seq): 
        x_test.append(inputs[i:(i+long_seq),0])
      x_test = np.array(x_test)
      x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

      y_pred = modelo.predict(x_test)
      y_pred = scaler.inverse_transform(y_pred)

      data_test = scaler.inverse_transform(data_test_scaled)

      rmse = sqrt(mean_squared_error(data_test, y_pred))
      rmses.append(rmse)

    medias.append(stats.mean(rmses))
    des_std.append(stats.stdev(rmses))

print("Las medias son:", medias)
print("Las desviaciones estandar son:",des_std)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
Epoch 29/180
45/45 [==============================] - 0s 4ms/step - loss: 0.0019
Epoch 30/180
45/45 [==============================] - 0s 4ms/step - loss: 0.0021
Epoch 31/180
45/45 [==============================] - 0s 4ms/step - loss: 0.0019
Epoch 32/180
45/45 [==============================] - 0s 4ms/step - loss: 0.0019
Epoch 33/180
45/45 [==============================] - 0s 4ms/step - loss: 0.0019
Epoch 34/180
45/45 [==============================] - 0s 4ms/step - loss: 0.0018
Epoch 35/180
45/45 [==============================] - 0s 4ms/step - loss: 0.0017
Epoch 36/180
45/45 [==============================] - 0s 7ms/step - loss: 0.0019
Epoch 37/180
45/45 [==============================] - 0s 7ms/step - loss: 0.0026
Epoch 38/180
45/45 [==============================] - 0s 7ms/step - loss: 0.0019
Epoch 39/180
45/45 [==============================] - 0s 6ms/step - loss: 0.0015
Epoch 40/180
45/45 [======================

In [ ]:
medias

[5932.211930690727,
 5725.356133331712,
 5377.995729534812,
 5098.151959914335,
 4886.057582375145,
 4927.194730227102,
 4914.618144169989,
 4822.041289856153,
 4753.125559478838,
 4686.077116458122]

In [ ]:
des_std

[4903.5426037298275,
 5011.208663260592,
 4775.877824574441,
 4535.914703870275,
 4289.242471545692,
 4306.477730043375,
 4255.992629969786,
 4149.417112992493,
 4113.024714649137,
 4039.2664769295093]